In [72]:
import requests
import os
from dotenv import load_dotenv
from datetime import datetime
import time
import json

# Accessing API Key Securely

In [7]:
'''
Securely Dealing with the API Key: First I need to learn about the riot apis and what information they
actually provide me so in order to do that I will store the API Key as an env variable on my os for now.
Once I actually start building the product I will use Hashi Corp Vault prod to do this.
'''
load_dotenv()
api_key = os.getenv('API_KEY')
print(api_key)

RGAPI-2a85fdbb-ab1c-45b9-b25e-20215ff4a6c2


# Understanding Workflow of Riot IDs

In [55]:
# There are platform routing vals and also regional routing vals as well, look at the reference example from the Riot Dev Api site to see which
# one does the url have
regional_routing_url = "https://americas.api.riotgames.com"

In [56]:
# Input: tagline, gameName, Output: json of puuid, tagline, gameName (ACCOUNT-V1)
# PUUID is a unique identifer that is specific to each player across all RIOT Games
gameName = "LemmeLastHit"
tagline = "5791"
puuid_url = f"{regional_routing_url}/riot/account/v1/accounts/by-riot-id/{gameName}/{tagline}?api_key={api_key}"
resp = requests.get(puuid_url)
data = resp.json()
print(data)

{'puuid': 'YiNBOnc_eNKyRiGMLrfEQ6d-s66XekPBprjgTl1_HFb8JvQiyW6hVt9hT7BLj92aC8UdSSy-Sq55ZQ', 'gameName': 'LemmeLastHit', 'tagLine': '5791'}


In [57]:
# Input: puuid, output: json of puuid, tagline, gameName (SUMMONER-V4)
puuid = data['puuid']
gameName_tagLine_url = f"{regional_routing_url}/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}"
resp = requests.get(puuid_url)
data = resp.json()
print(data)

{'puuid': 'YiNBOnc_eNKyRiGMLrfEQ6d-s66XekPBprjgTl1_HFb8JvQiyW6hVt9hT7BLj92aC8UdSSy-Sq55ZQ', 'gameName': 'LemmeLastHit', 'tagLine': '5791'}


In [58]:
riotId_from_puuid_url = f"{regional_routing_url}/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}"
resp = requests.get(riotId_from_puuid_url)
data = resp.json()
print(data)

{'puuid': 'YiNBOnc_eNKyRiGMLrfEQ6d-s66XekPBprjgTl1_HFb8JvQiyW6hVt9hT7BLj92aC8UdSSy-Sq55ZQ', 'gameName': 'LemmeLastHit', 'tagLine': '5791'}


### Match Data 

In [59]:
# Aggregate Match ID's for a specific player do this from RiotID
gameName = "LemmeLastHit"
tagline = "5791"
puuid_url = f"{regional_routing_url}/riot/account/v1/accounts/by-riot-id/{gameName}/{tagline}?api_key={api_key}"
resp = requests.get(puuid_url)
data = resp.json() # contains the puuid

{'puuid': 'YiNBOnc_eNKyRiGMLrfEQ6d-s66XekPBprjgTl1_HFb8JvQiyW6hVt9hT7BLj92aC8UdSSy-Sq55ZQ', 'gameName': 'LemmeLastHit', 'tagLine': '5791'}


In [66]:
def human_date_to_epoch(year, month, day, hour=0, minute=0, second=0):
    dt = datetime(year, month, day, hour, minute, second)
    timestamp = int(time.mktime(dt.timetuple()))
    return timestamp

endTime = human_date_to_epoch(2024, 4, 23)
startTime = human_date_to_epoch(2023, 4, 23)
queue = 420
type = "ranked"
start = 0
count = 100

# print(startTime)
# print(endTime)

aggregate_matches_url = f"{regional_routing_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?queue={queue}&type={type}&start={start}&count={count}&api_key={api_key}"
resp = requests.get(aggregate_matches_url)
matchIds = resp.json() 
# print(matchIds)

['NA1_4981409911', 'NA1_4980923487', 'NA1_4977215784', 'NA1_4977071623', 'NA1_4976995986', 'NA1_4976810761', 'NA1_4976448607', 'NA1_4976264924', 'NA1_4974928949', 'NA1_4974698313', 'NA1_4974565860', 'NA1_4974228695', 'NA1_4973815844', 'NA1_4973788080', 'NA1_4973622938', 'NA1_4973330751', 'NA1_4973276476', 'NA1_4972950893', 'NA1_4971482693', 'NA1_4971451861', 'NA1_4970796047', 'NA1_4970280543', 'NA1_4969947975', 'NA1_4969591951', 'NA1_4969574148', 'NA1_4969119101', 'NA1_4969089121', 'NA1_4968748740', 'NA1_4968244016', 'NA1_4967911629', 'NA1_4967888285', 'NA1_4967152782', 'NA1_4967117791', 'NA1_4967026396', 'NA1_4966965548', 'NA1_4966024446', 'NA1_4965736834', 'NA1_4965599690', 'NA1_4963680148', 'NA1_4963218098', 'NA1_4963043581', 'NA1_4962261930', 'NA1_4955900298', 'NA1_4955818514', 'NA1_4955731490', 'NA1_4954594743', 'NA1_4953859247', 'NA1_4953679341', 'NA1_4952876521', 'NA1_4951808372', 'NA1_4951023905', 'NA1_4950997899', 'NA1_4948947352', 'NA1_4948919128', 'NA1_4948830284', 'NA1_4946

In [74]:
# Access statistics from one of the matches
matchId = matchIds[-1]
match_results_url = f"{regional_routing_url}/lol/match/v5/matches/{matchId}?api_key={api_key}"
resp = requests.get(match_results_url)

if resp.status_code == 200:
	match_data = resp.json()
	filename = f'match_results_{matchId}.json'
	with open(filename, 'w') as f:
		json.dump(match_data, f, indent=4)